In [1]:
# Model Training
from datasets import load_from_disk

datasets = load_from_disk("./encoded_data3_better")
datasets

c:\Users\user\Desktop\QuestionQuestionModel\venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    train: Dataset({
        features: ['String', 'Algorithm Design', 'Basic Machine Organisation', 'Computer System', 'Data Manipulation and Analysis', 'Data Organisation and Data Control', 'Elementary Web Authoring', 'Health and Ethical Issues', 'Information Processing', 'Intellectual Property', 'Internet Services and Applications', 'Multimedia Elements', 'Networking and Internet Basics', 'Program Development', 'Spreadsheets and Databases', 'Threats and Security on the Internet', '__index_level_0__'],
        num_rows: 648
    })
    valid: Dataset({
        features: ['String', 'Algorithm Design', 'Basic Machine Organisation', 'Computer System', 'Data Manipulation and Analysis', 'Data Organisation and Data Control', 'Elementary Web Authoring', 'Health and Ethical Issues', 'Information Processing', 'Intellectual Property', 'Internet Services and Applications', 'Multimedia Elements', 'Networking and Internet Basics', 'Program Development', 'Spreadsheets and Databases', '

In [2]:
model_labels = [label for label in datasets['train'].features.keys() if label not in ['Unnamed: 0', 'String', '__index_level_0__']]

In [3]:
datasets["train"].column_names

['String',
 'Algorithm Design',
 'Basic Machine Organisation',
 'Computer System',
 'Data Manipulation and Analysis',
 'Data Organisation and Data Control',
 'Elementary Web Authoring',
 'Health and Ethical Issues',
 'Information Processing',
 'Intellectual Property',
 'Internet Services and Applications',
 'Multimedia Elements',
 'Networking and Internet Basics',
 'Program Development',
 'Spreadsheets and Databases',
 'Threats and Security on the Internet',
 '__index_level_0__']

In [4]:
datasets["train"].features

{'String': Value(dtype='string', id=None),
 'Algorithm Design': Value(dtype='int64', id=None),
 'Basic Machine Organisation': Value(dtype='int64', id=None),
 'Computer System': Value(dtype='int64', id=None),
 'Data Manipulation and Analysis': Value(dtype='int64', id=None),
 'Data Organisation and Data Control': Value(dtype='int64', id=None),
 'Elementary Web Authoring': Value(dtype='int64', id=None),
 'Health and Ethical Issues': Value(dtype='int64', id=None),
 'Information Processing': Value(dtype='int64', id=None),
 'Intellectual Property': Value(dtype='int64', id=None),
 'Internet Services and Applications': Value(dtype='int64', id=None),
 'Multimedia Elements': Value(dtype='int64', id=None),
 'Networking and Internet Basics': Value(dtype='int64', id=None),
 'Program Development': Value(dtype='int64', id=None),
 'Spreadsheets and Databases': Value(dtype='int64', id=None),
 'Threats and Security on the Internet': Value(dtype='int64', id=None),
 '__index_level_0__': Value(dtype='int64

In [5]:
#print(f"{datasets.train.column_names}\n{datasets.train.features}")

In [6]:
model_labels

['Algorithm Design',
 'Basic Machine Organisation',
 'Computer System',
 'Data Manipulation and Analysis',
 'Data Organisation and Data Control',
 'Elementary Web Authoring',
 'Health and Ethical Issues',
 'Information Processing',
 'Intellectual Property',
 'Internet Services and Applications',
 'Multimedia Elements',
 'Networking and Internet Basics',
 'Program Development',
 'Spreadsheets and Databases',
 'Threats and Security on the Internet']

In [7]:
id2label = {idx:label for idx, label in enumerate(model_labels)}
label2id = {label:idx for idx, label in enumerate(model_labels)}

In [8]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
import numpy as np

tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

In [9]:
def encode_data(dataset):
    text = dataset["String"]
    # tokenize string
    encoding = tokenizer(text, padding="max_length", truncation=True, max_length=70)
    # create encoded array list with labels
    labelsBatch = {x: dataset[x] for x in dataset.keys() if x in model_labels}
    labels_matrix = np.zeros((len(text), len(model_labels)))
    for idx, label in enumerate(model_labels):
        labels_matrix[:, idx] = labelsBatch[label]
    
    encoding["labels"] = labels_matrix.tolist()
    return encoding

In [10]:
# tokenize the dataset
encoded_ds = datasets.map(encode_data, batched=True, remove_columns=datasets['train'].column_names)

Loading cached processed dataset at c:\Users\user\Desktop\QuestionQuestionModel\model\encoded_data3_better\train\cache-1668182f3c08226f.arrow
Loading cached processed dataset at c:\Users\user\Desktop\QuestionQuestionModel\model\encoded_data3_better\valid\cache-339e07acb2223203.arrow
Loading cached processed dataset at c:\Users\user\Desktop\QuestionQuestionModel\model\encoded_data3_better\test\cache-1616b207cb5722c2.arrow


In [11]:
encoded_ds['train']['labels'][0]

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]

In [12]:
[id2label[idx] for idx, label in enumerate(encoded_ds['train'][15]['labels']) if label == 1.0]

['Data Organisation and Data Control']

In [13]:
import torch
from torch import cuda
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

encoded_ds.set_format("torch")

Using device: cuda



In [14]:
encoded_ds["train"]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 648
})

In [15]:

# define model
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", problem_type="multi_label_classification", num_labels=len(model_labels), id2label=id2label, label2id=label2id)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

In [16]:
from transformers import AdamW
optimizer = AdamW(model.parameters(), lr=5e-5)

c:\Users\user\Desktop\QuestionQuestionModel\venv\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [17]:
from transformers import get_scheduler

num_training_steps = 2430
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)
print(num_training_steps)

2430


In [18]:
# train model
batch_size = 8
metric_name = "f1"

In [19]:
import evaluate
metric  = evaluate.load("accuracy")

In [20]:
import time

train_name_dir = time.strftime(f"%Y%m%d_%H%M%S$-train-model-{model.name_or_path}")
train_name_dir

'20230312_044105$-train-model-distilbert-base-uncased'

In [21]:
from transformers import TrainingArguments, Trainer


args = TrainingArguments(
    output_dir="./distBert-v8/"+ train_name_dir,
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs= 81,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    logging_dir='./distBert-v8/logging/'+ train_name_dir
)

In [22]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction


def multi_label_metrics(predictions, labels, threshold=0.50):
    # apply sigmoid on predictions fitting (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # logits, labels = predictions
    # prediction = np.argmax(logits, axis=-1)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result

def compute_metrics_new(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [23]:
encoded_ds['train'][0]['labels'].type()

'torch.FloatTensor'

In [24]:
# model.to(device)

In [25]:
outputs = model(input_ids=encoded_ds['train']['input_ids'][0].unsqueeze(0), labels=encoded_ds['train'][0]['labels'].unsqueeze(0))
outputs

SequenceClassifierOutput(loss=tensor(0.6815, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), logits=tensor([[-0.0507, -0.0173, -0.0950, -0.0450, -0.0969,  0.0082,  0.1329,  0.0231,
         -0.0566, -0.1218,  0.0390, -0.0533, -0.0410,  0.0573,  0.0652]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [26]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_ds["train"],
    eval_dataset=encoded_ds["valid"],
    compute_metrics=compute_metrics,
    tokenizer=tokenizer
)

In [27]:
trainer.train()

c:\Users\user\Desktop\QuestionQuestionModel\venv\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 648
  Num Epochs = 81
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 6561
  Number of trainable parameters = 66965007
  1%|          | 81/6561 [00:05<05:45, 18.77it/s] ***** Running Evaluation *****
  Num examples = 81
  Batch size = 8
                                                 
  1%|          | 81/6561 [00:06<05:45, 18.77it/s]Saving model checkpoint to ./distBert-v8/20230312_044105$-train-model-distilbert-base-uncased\checkpoint-81
Configuration saved in ./distBert-v8/20230312_

{'eval_loss': 0.27346426248550415, 'eval_f1': 0.0, 'eval_roc_auc': 0.5, 'eval_accuracy': 0.0, 'eval_runtime': 0.1491, 'eval_samples_per_second': 543.431, 'eval_steps_per_second': 73.799, 'epoch': 1.0}


Model weights saved in ./distBert-v8/20230312_044105$-train-model-distilbert-base-uncased\checkpoint-81\pytorch_model.bin
tokenizer config file saved in ./distBert-v8/20230312_044105$-train-model-distilbert-base-uncased\checkpoint-81\tokenizer_config.json
Special tokens file saved in ./distBert-v8/20230312_044105$-train-model-distilbert-base-uncased\checkpoint-81\special_tokens_map.json
  2%|▏         | 161/6561 [00:19<05:42, 18.71it/s] ***** Running Evaluation *****
  Num examples = 81
  Batch size = 8
                                                  
  2%|▏         | 162/6561 [00:19<05:42, 18.71it/s]Saving model checkpoint to ./distBert-v8/20230312_044105$-train-model-distilbert-base-uncased\checkpoint-162
Configuration saved in ./distBert-v8/20230312_044105$-train-model-distilbert-base-uncased\checkpoint-162\config.json


{'eval_loss': 0.2528999149799347, 'eval_f1': 0.0, 'eval_roc_auc': 0.5, 'eval_accuracy': 0.0, 'eval_runtime': 0.1448, 'eval_samples_per_second': 559.502, 'eval_steps_per_second': 75.982, 'epoch': 2.0}


Model weights saved in ./distBert-v8/20230312_044105$-train-model-distilbert-base-uncased\checkpoint-162\pytorch_model.bin
tokenizer config file saved in ./distBert-v8/20230312_044105$-train-model-distilbert-base-uncased\checkpoint-162\tokenizer_config.json
Special tokens file saved in ./distBert-v8/20230312_044105$-train-model-distilbert-base-uncased\checkpoint-162\special_tokens_map.json
  4%|▎         | 243/6561 [00:25<05:36, 18.79it/s]***** Running Evaluation *****
  Num examples = 81
  Batch size = 8
                                                  
  4%|▎         | 243/6561 [00:25<05:36, 18.79it/s]Saving model checkpoint to ./distBert-v8/20230312_044105$-train-model-distilbert-base-uncased\checkpoint-243
Configuration saved in ./distBert-v8/20230312_044105$-train-model-distilbert-base-uncased\checkpoint-243\config.json


...

100%|█████████▉| 6559/6561 [16:01<00:00, 20.12it/s]***** Running Evaluation *****
  Num examples = 81
  Batch size = 8
                                                   
100%|██████████| 6561/6561 [16:01<00:00, 20.12it/s]Saving model checkpoint to ./distBert-v8/20230312_044105$-train-model-distilbert-base-uncased\checkpoint-6561
Configuration saved in ./distBert-v8/20230312_044105$-train-model-distilbert-base-uncased\checkpoint-6561\config.json
100%|██████████| 6561/6561 [16:12<00:00, 20.12it/s]

{'eval_loss': 0.13299304246902466, 'eval_f1': 0.8072289156626506, 'eval_roc_auc': 0.8837775763692917, 'eval_accuracy': 0.7530864197530864, 'eval_runtime': 0.1367, 'eval_samples_per_second': 592.745, 'eval_steps_per_second': 80.496, 'epoch': 81.0}


Model weights saved in ./distBert-v8/20230312_044105$-train-model-distilbert-base-uncased\checkpoint-6561\pytorch_model.bin
tokenizer config file saved in ./distBert-v8/20230312_044105$-train-model-distilbert-base-uncased\checkpoint-6561\tokenizer_config.json
Special tokens file saved in ./distBert-v8/20230312_044105$-train-model-distilbert-base-uncased\checkpoint-6561\special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from ./distBert-v8/20230312_044105$-train-model-distilbert-base-uncased\checkpoint-3564 (score: 0.8242424242424243).
100%|██████████| 6561/6561 [16:14<00:00,  6.74it/s]

{'train_runtime': 974.1846, 'train_samples_per_second': 53.879, 'train_steps_per_second': 6.735, 'train_loss': 0.02326273483364316, 'epoch': 81.0}


TrainOutput(global_step=6561, training_loss=0.02326273483364316, metrics={'train_runtime': 974.1846, 'train_samples_per_second': 53.879, 'train_steps_per_second': 6.735, 'train_loss': 0.02326273483364316, 'epoch': 81.0})

In [28]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 81
  Batch size = 8
100%|██████████| 11/11 [00:00<00:00, 60.92it/s]


{'eval_loss': 0.11271076649427414,
 'eval_f1': 0.8242424242424243,
 'eval_roc_auc': 0.8904772694502235,
 'eval_accuracy': 0.7777777777777778,
 'eval_runtime': 0.1985,
 'eval_samples_per_second': 408.052,
 'eval_steps_per_second': 55.414,
 'epoch': 81.0}

In [29]:
prediction_result = trainer.predict(encoded_ds['test'])

***** Running Prediction *****
  Num examples = 81
  Batch size = 8
100%|██████████| 11/11 [00:00<00:00, 98.87it/s]


In [30]:
print(prediction_result.metrics)

{'test_loss': 0.14533638954162598, 'test_f1': 0.7354838709677419, 'test_roc_auc': 0.8405000753449723, 'test_accuracy': 0.6666666666666666, 'test_runtime': 0.1272, 'test_samples_per_second': 636.718, 'test_steps_per_second': 86.468}


In [31]:
trainer.save_model("./distlBert-model-v8")

Saving model checkpoint to ./distlBert-model-v8
Configuration saved in ./distlBert-model-v8\config.json
Model weights saved in ./distlBert-model-v8\pytorch_model.bin
tokenizer config file saved in ./distlBert-model-v8\tokenizer_config.json
Special tokens file saved in ./distlBert-model-v8\special_tokens_map.json
